## New experiment  for different channel permutation
### choose fro

In [ ]:
from addict import Dict
import geopandas as gpd
import pathlib
import yaml
import numpy as np
import rasterio
import glacier_mapping.data.process_slices_funs as pf

In [2]:
# number of experiment
# start from 0 up to 11, with 0 being [5, 4, 2]
i = 1

In [3]:
output_dir = pathlib.Path("/mnt/blobfuse/glaciers/processed")
pconf = Dict(yaml.safe_load(open(f"/mnt/blobfuse/glaciers/conf/postprocess_exper{i}.yaml", "r")))
slice_meta = gpd.read_file(pathlib.Path("/mnt/blobfuse/glaciers/processed", "slices", "slices.geojson"))

In [26]:
#read targlocs so we use exactly same data for every experiment
import pickle

with open("/mnt/blobfuse/glaciers/processed/target_locs.pkl", "rb") as f:
        target_locs = pickle.load(f)

In [5]:
# post process
import os

print("getting stats")
data_dir = pathlib.Path("/mnt/blobfuse/glaciers")
processed_dir = data_dir / "processed"
pconf.process_funs.normalize.stats_path = processed_dir / \
    pathlib.Path(pconf.process_funs.normalize.stats_path)

stats = pf.generate_stats(
    [p["img"] for p in target_locs["train"]],
    pconf.normalization_sample_size,
    pconf.process_funs.normalize.stats_path,
)

# make new dirs
for split_type in target_locs:
    new_path = target_locs[split_type][0]["img"].parent.parent
    os.makedirs(new_path / f'{split_type}_{i}', exist_ok=True)
    
# postprocess individual images (all the splits)
for split_type in target_locs:
    print(f"postprocessing {split_type}...")
    for k in range(len(target_locs[split_type])):
        img, mask = pf.postprocess(
            target_locs[split_type][k]["img"],
            target_locs[split_type][k]["mask"],
            pconf.process_funs,
        )
        
        new_img_loc = str(target_locs[split_type][k]["img"]).replace(split_type, f'{split_type}_{i}')
        new_mask_loc = str(target_locs[split_type][k]["mask"]).replace(split_type, f'{split_type}_{i}')
        np.save(pathlib.Path(new_img_loc), img)
        np.save(pathlib.Path(new_mask_loc), mask)


getting stats
postprocessing train...
postprocessing dev...
postprocessing test...


In [6]:
from glacier_mapping.data.data import fetch_loaders
from glacier_mapping.models.frame import Framework
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
import torch
import json

data_dir = pathlib.Path("/mnt/blobfuse/glaciers")
conf = Dict(yaml.safe_load(open(f"/mnt/blobfuse/glaciers/conf/train_exper{i}.yaml", "r")))
processed_dir = data_dir / "processed"

args = Dict({
    "batch_size": 16,
    "run_name": f"run_exper_{i}",
    "epochs": 201,
    "save_every": 50
})

loaders = fetch_loaders(processed_dir, args.batch_size, train_folder=f"train_{i}", dev_folder=f"dev_{i}")
frame = Framework(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts
)

# Setup logging
writer = SummaryWriter(f"{data_dir}/runs/{args.run_name}/logs/")
writer.add_text("Arguments", json.dumps(vars(args)))
writer.add_text("Configuration Parameters", json.dumps(conf))
out_dir = f"{data_dir}/runs/{args.run_name}/models/"

for epoch in range(args.epochs):

    # train loop
    loss_d = {}
    loss_d["train"], metrics = tr.train_epoch(loaders["train"], frame, conf.metrics_opts)
    tr.log_metrics(writer, metrics, loss_d["train"], epoch)
    tr.log_images(writer, frame, next(iter(loaders["train"])), epoch)

    # validation loop
    loss_d["val"], metrics = tr.validate(loaders["val"], frame, conf.metrics_opts)
    tr.log_metrics(writer, metrics, loss_d["val"], epoch, "val")
    tr.log_images(writer, frame, next(iter(loaders["val"])), epoch, "val")

    # Save model"
    writer.add_scalars("Loss", loss_d, epoch)
    if epoch % args.save_every == 0:
        frame.save(out_dir, epoch)

    print(f"{epoch}/{args.epochs} | train: {loss_d['train']} | val: {loss_d['val']}")

frame.save(out_dir, "final")
writer.close()

KeyboardInterrupt: 